# E-Commerce Business Analytics Dashboard

This notebook provides a comprehensive analysis of e-commerce business performance, including revenue trends, customer behavior, product performance, and operational metrics.

## Table of Contents

1. [Introduction and Business Objectives](#introduction)
2. [Data Dictionary](#data-dictionary)
3. [Configuration](#configuration)
4. [Data Loading and Preparation](#data-loading)
5. [Business Metrics Analysis](#business-metrics)
   - 5.1 [Revenue Analysis](#revenue-analysis)
   - 5.2 [Product Performance Analysis](#product-analysis)
   - 5.3 [Geographic Analysis](#geographic-analysis)
   - 5.4 [Customer Experience Analysis](#customer-experience)
6. [Summary and Key Observations](#summary)

<a id='introduction'></a>
## 1. Introduction and Business Objectives

### Business Objectives

This analysis aims to answer the following key business questions:

1. How has revenue performed compared to the previous period?
2. What are the month-over-month growth trends?
3. Which product categories are generating the most revenue?
4. How does geographic location impact sales performance?
5. How does delivery speed affect customer satisfaction?
6. What is the overall customer experience as measured by review scores?

### Analysis Period

The analysis focuses on a configurable date range, allowing for flexible period-over-period comparisons.

<a id='data-dictionary'></a>
## 2. Data Dictionary

### Key Datasets

**Orders Dataset**
- `order_id`: Unique identifier for each order
- `customer_id`: Unique identifier for each customer
- `order_status`: Current status (delivered, shipped, canceled, processing, pending, returned)
- `order_purchase_timestamp`: Date and time when order was placed
- `order_delivered_customer_date`: Date and time when order was delivered to customer

**Order Items Dataset**
- `order_id`: Links to orders
- `product_id`: Links to products
- `price`: Item price in dollars
- `freight_value`: Shipping cost in dollars

**Products Dataset**
- `product_id`: Unique identifier for each product
- `product_category_name`: Product category (13 categories total)

**Customers Dataset**
- `customer_id`: Unique identifier
- `customer_state`: US state abbreviation
- `customer_city`: City name

**Reviews Dataset**
- `review_id`: Unique identifier for review
- `order_id`: Links to orders
- `review_score`: Rating from 1 to 5

### Business Metrics Definitions

- **Revenue**: Sum of all order item prices (excludes freight)
- **Average Order Value (AOV)**: Total revenue divided by number of unique orders
- **Month-over-Month (MoM) Growth**: Percentage change in revenue from one month to the next
- **Delivery Speed**: Number of days from order purchase to customer delivery
- **Review Score**: Customer satisfaction rating on a scale of 1-5

<a id='configuration'></a>
## 3. Configuration

Configure the analysis parameters below to analyze different time periods.

In [ ]:
# Analysis Configuration
# Adjust these parameters to analyze different time periods

# Current period (e.g., 2023 full year)
CURRENT_START_YEAR = 2023
CURRENT_START_MONTH = 1
CURRENT_END_YEAR = 2023
CURRENT_END_MONTH = 12

# Comparison period (e.g., 2022 full year)
COMPARISON_START_YEAR = 2022
COMPARISON_START_MONTH = 1
COMPARISON_END_YEAR = 2022
COMPARISON_END_MONTH = 12

# Data path (leave empty if CSV files are in current directory)
DATA_PATH = ''

# Visualization settings
CHART_COLOR_PRIMARY = '#2E86AB'
CHART_COLOR_SECONDARY = '#A23B72'
CHART_COLOR_ACCENT = '#F18F01'
FIGURE_SIZE = (12, 6)

<a id='data-loading'></a>
## 4. Data Loading and Preparation

Load all datasets and prepare them for analysis.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import warnings

# Import custom modules
import data_loader as dl
import business_metrics as bm

# Configuration
warnings.filterwarnings('ignore')
pd.set_option('display.float_format', '{:.2f}'.format)

print("Libraries loaded successfully.")

In [ ]:
# Load all datasets
print("Loading datasets...")
datasets = dl.load_datasets(data_path=DATA_PATH)

# Extract individual datasets
orders = datasets['orders']
order_items = datasets['order_items']
products = datasets['products']
customers = datasets['customers']
reviews = datasets['reviews']

print(f"Loaded {len(orders):,} orders")
print(f"Loaded {len(order_items):,} order items")
print(f"Loaded {len(products):,} products")
print(f"Loaded {len(customers):,} customers")
print(f"Loaded {len(reviews):,} reviews")

In [ ]:
# Prepare datasets with datetime conversions
print("Preparing datasets...")
orders = dl.prepare_orders_data(orders)
reviews = dl.prepare_reviews_data(reviews)

print("Data preparation complete.")
print(f"\nDate range in data: {orders['order_purchase_timestamp'].min().date()} to {orders['order_purchase_timestamp'].max().date()}")

In [ ]:
# Create consolidated sales dataset for current period
print(f"\nCreating sales dataset for current period: {CURRENT_START_YEAR}-{CURRENT_START_MONTH:02d} to {CURRENT_END_YEAR}-{CURRENT_END_MONTH:02d}")

# Get all delivered orders
sales_all = dl.create_sales_dataset(order_items, orders, status_filter='delivered')

# Filter to current period
sales_current = dl.filter_by_date_range(
    sales_all,
    start_year=CURRENT_START_YEAR,
    start_month=CURRENT_START_MONTH,
    end_year=CURRENT_END_YEAR,
    end_month=CURRENT_END_MONTH
)

# Filter to comparison period
sales_comparison = dl.filter_by_date_range(
    sales_all,
    start_year=COMPARISON_START_YEAR,
    start_month=COMPARISON_START_MONTH,
    end_year=COMPARISON_END_YEAR,
    end_month=COMPARISON_END_MONTH
)

print(f"Current period: {len(sales_current):,} delivered order items")
print(f"Comparison period: {len(sales_comparison):,} delivered order items")

In [ ]:
# Enrich sales data with additional dimensions
print("\nEnriching sales data with product categories...")
sales_current = dl.add_product_categories(sales_current, products)
sales_comparison = dl.add_product_categories(sales_comparison, products)

print("Enriching sales data with customer geography...")
sales_current = dl.add_customer_geography(sales_current, orders, customers)
sales_comparison = dl.add_customer_geography(sales_comparison, orders, customers)

print("Enriching sales data with review scores...")
sales_current = dl.add_review_scores(sales_current, reviews)
sales_comparison = dl.add_review_scores(sales_comparison, reviews)

print("Calculating delivery speed...")
sales_current = dl.calculate_delivery_speed(sales_current)
sales_comparison = dl.calculate_delivery_speed(sales_comparison)

# Add delivery categories
sales_current['delivery_category'] = sales_current['delivery_speed_days'].apply(dl.categorize_delivery_speed)
sales_comparison['delivery_category'] = sales_comparison['delivery_speed_days'].apply(dl.categorize_delivery_speed)

print("Data enrichment complete.")

<a id='business-metrics'></a>
## 5. Business Metrics Analysis

<a id='revenue-analysis'></a>
### 5.1 Revenue Analysis

Analyze overall revenue performance, comparing current period to previous period.

In [ ]:
# Calculate period comparison metrics
comparison_metrics = bm.compare_periods(sales_current, sales_comparison)

# Display results
print("=" * 60)
print("REVENUE COMPARISON")
print("=" * 60)
print(f"\nCurrent Period ({CURRENT_START_YEAR})")
print(f"  Total Revenue: ${comparison_metrics['current_revenue']:,.2f}")
print(f"  Total Orders: {comparison_metrics['current_orders']:,.0f}")
print(f"  Average Order Value: ${comparison_metrics['current_aov']:,.2f}")

print(f"\nComparison Period ({COMPARISON_START_YEAR})")
print(f"  Total Revenue: ${comparison_metrics['previous_revenue']:,.2f}")
print(f"  Total Orders: {comparison_metrics['previous_orders']:,.0f}")
print(f"  Average Order Value: ${comparison_metrics['previous_aov']:,.2f}")

print(f"\nYear-over-Year Growth")
print(f"  Revenue Growth: {comparison_metrics['revenue_growth_rate']*100:+.2f}%")
print(f"  Orders Growth: {comparison_metrics['orders_growth_rate']*100:+.2f}%")
print(f"  AOV Growth: {comparison_metrics['aov_growth_rate']*100:+.2f}%")
print("=" * 60)

#### Monthly Revenue Trend

Visualize month-over-month revenue performance for the current period.

In [ ]:
# Calculate monthly revenue
monthly_revenue = bm.calculate_revenue_by_period(sales_current, period='year-month')

# Create visualization
plt.figure(figsize=FIGURE_SIZE)
plt.plot(monthly_revenue['month'], monthly_revenue['revenue'], 
         marker='o', linewidth=2, markersize=8, color=CHART_COLOR_PRIMARY)
plt.title(f'Monthly Revenue Trend - {CURRENT_START_YEAR}', fontsize=14, fontweight='bold', pad=20)
plt.xlabel('Month', fontsize=12)
plt.ylabel('Revenue (USD)', fontsize=12)
plt.grid(True, alpha=0.3)
plt.xticks(range(1, 13), ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
                          'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])

# Format y-axis as currency
ax = plt.gca()
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1000:.0f}K'))

plt.tight_layout()
plt.show()

#### Month-over-Month Growth Analysis

Analyze the growth rate from month to month.

In [ ]:
# Calculate MoM growth
monthly_growth = bm.calculate_mom_growth(monthly_revenue)

# Display growth table
print(f"\nMonth-over-Month Growth - {CURRENT_START_YEAR}")
print("=" * 50)
display_cols = monthly_growth[['month', 'revenue', 'mom_growth']].copy()
display_cols['revenue'] = display_cols['revenue'].apply(lambda x: f"${x:,.2f}")
display_cols['mom_growth'] = display_cols['mom_growth'].apply(
    lambda x: f"{x*100:+.2f}%" if pd.notna(x) else "N/A"
)
display_cols.columns = ['Month', 'Revenue', 'MoM Growth']
print(display_cols.to_string(index=False))

# Calculate average MoM growth (excluding first month)
avg_mom_growth = monthly_growth['mom_growth'].mean()
print("=" * 50)
print(f"Average MoM Growth: {avg_mom_growth*100:+.2f}%")

<a id='product-analysis'></a>
### 5.2 Product Performance Analysis

Analyze revenue contribution by product category to identify top performers.

In [ ]:
# Calculate revenue by category
category_revenue = bm.calculate_revenue_by_category(sales_current)

# Display top categories
print(f"\nTop Product Categories by Revenue - {CURRENT_START_YEAR}")
print("=" * 60)
category_display = category_revenue.copy()
category_display['revenue_formatted'] = category_display['revenue'].apply(lambda x: f"${x:,.2f}")
category_display['percentage'] = (category_display['revenue'] / category_display['revenue'].sum() * 100).apply(lambda x: f"{x:.1f}%")
print(category_display[['category', 'revenue_formatted', 'percentage']].to_string(index=False))
print("=" * 60)

#### Revenue by Product Category Visualization

In [ ]:
# Create bar chart
plt.figure(figsize=FIGURE_SIZE)
plt.barh(category_revenue['category'], category_revenue['revenue'], color=CHART_COLOR_PRIMARY)
plt.title(f'Revenue by Product Category - {CURRENT_START_YEAR}', fontsize=14, fontweight='bold', pad=20)
plt.xlabel('Revenue (USD)', fontsize=12)
plt.ylabel('Product Category', fontsize=12)
plt.grid(True, alpha=0.3, axis='x')

# Format x-axis as currency
ax = plt.gca()
ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1000:.0f}K'))

plt.tight_layout()
plt.show()

<a id='geographic-analysis'></a>
### 5.3 Geographic Analysis

Analyze sales performance across different US states.

In [ ]:
# Calculate revenue by state
state_revenue = bm.calculate_revenue_by_state(sales_current)

# Display top 10 states
print(f"\nTop 10 States by Revenue - {CURRENT_START_YEAR}")
print("=" * 50)
top_states = state_revenue.head(10).copy()
top_states['revenue_formatted'] = top_states['revenue'].apply(lambda x: f"${x:,.2f}")
top_states['percentage'] = (top_states['revenue'] / state_revenue['revenue'].sum() * 100).apply(lambda x: f"{x:.1f}%")
print(top_states[['state', 'revenue_formatted', 'percentage']].to_string(index=False))
print("=" * 50)

#### Geographic Distribution - Choropleth Map

In [ ]:
# Create complete list of all US state abbreviations
all_us_states = pd.DataFrame({
    'state': [
        'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA',
        'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD',
        'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ',
        'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC',
        'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY', 'DC'
    ]
})

# Merge with actual revenue data (left join to keep all states)
state_revenue_complete = all_us_states.merge(state_revenue, on='state', how='left')

# Replace NaN with 0 for states without data
state_revenue_complete['revenue'] = state_revenue_complete['revenue'].fillna(0)

# Add a display column for better hover information
state_revenue_complete['revenue_display'] = state_revenue_complete['revenue'].apply(
    lambda x: 'No data' if x == 0 else f'${x:,.2f}'
)

# For visualization, use a tiny value instead of 0 to ensure rendering
state_revenue_complete['revenue_viz'] = state_revenue_complete['revenue'].replace(0, 0.001)

# Get the actual max revenue
max_revenue = state_revenue_complete[state_revenue_complete['revenue'] > 0]['revenue'].max() if len(state_revenue_complete[state_revenue_complete['revenue'] > 0]) > 0 else 1000000

print(f"Total states in dataset: {len(state_revenue_complete)}")
print(f"States with revenue: {len(state_revenue_complete[state_revenue_complete['revenue'] > 0])}")
print(f"States without revenue: {len(state_revenue_complete[state_revenue_complete['revenue'] == 0])}")

# Create choropleth map with dark background
fig = px.choropleth(
    state_revenue_complete,
    locations='state',
    color='revenue_viz',
    locationmode='USA-states',
    scope='usa',
    title=f'Revenue by State - {CURRENT_START_YEAR}',
    color_continuous_scale=[
        [0, '#CCCCCC'],          # Light gray for no data
        [0.0001, '#CCCCCC'],     # Keep light gray 
        [0.001, '#C6DBEF'],      # Very light blue
        [0.1, '#9ECAE1'],        # Light blue
        [0.3, '#6BAED6'],        # Medium blue
        [0.6, '#3182BD'],        # Blue
        [1.0, '#08519C']         # Dark blue
    ],
    labels={'revenue_viz': 'Revenue (USD)'},
    range_color=[0, max_revenue],
    hover_data={'revenue_viz': False, 'revenue_display': True, 'state': True}
)

fig.update_traces(
    marker_line_color='#FFFFFF',
    marker_line_width=1.5,
    hovertemplate='<b>State: %{location}</b><br>Revenue: %{customdata[0]}<extra></extra>'
)

fig.update_layout(
    title_font_size=16,
    title_font_family='Arial',
    title_font_color='#FFFFFF',
    height=500,
    paper_bgcolor='#2C3E50',
    plot_bgcolor='#2C3E50',
    font=dict(color='#FFFFFF'),
    geo=dict(
        bgcolor='#34495E',
        lakecolor='#1A252F',
        showlakes=True,
        projection_type='albers usa'
    ),
    coloraxis_colorbar=dict(
        title='Revenue (USD)',
        tickformat='$,.0f'
    )
)

fig.show()

<a id='customer-experience'></a>
### 5.4 Customer Experience Analysis

Analyze customer satisfaction through review scores and delivery performance.

#### Order Status Distribution

In [ ]:
# Filter orders for current period
orders_current = dl.filter_by_date_range(
    orders,
    start_year=CURRENT_START_YEAR,
    start_month=CURRENT_START_MONTH,
    end_year=CURRENT_END_YEAR,
    end_month=CURRENT_END_MONTH
)

# Calculate status distribution
status_dist = bm.calculate_order_status_distribution(orders_current)

print(f"\nOrder Status Distribution - {CURRENT_START_YEAR}")
print("=" * 50)
status_display = status_dist.copy()
status_display['percentage_formatted'] = status_display['percentage'].apply(lambda x: f"{x:.1f}%")
print(status_display[['order_status', 'count', 'percentage_formatted']].to_string(index=False))
print("=" * 50)

#### Review Score Analysis

In [ ]:
# Calculate review metrics
avg_review_score = bm.calculate_average_review_score(sales_current)
review_distribution = bm.calculate_review_score_distribution(sales_current)

print(f"\nCustomer Review Analysis - {CURRENT_START_YEAR}")
print("=" * 50)
print(f"Average Review Score: {avg_review_score:.2f} out of 5.00")
print("\nReview Score Distribution:")
print(review_distribution.to_string(index=False))
print("=" * 50)

In [ ]:
# Visualize review score distribution
plt.figure(figsize=(10, 6))
plt.barh(review_distribution['review_score'].astype(str), 
         review_distribution['percentage'],
         color=CHART_COLOR_PRIMARY)
plt.title(f'Review Score Distribution - {CURRENT_START_YEAR}', fontsize=14, fontweight='bold', pad=20)
plt.xlabel('Percentage of Reviews (%)', fontsize=12)
plt.ylabel('Review Score', fontsize=12)
plt.grid(True, alpha=0.3, axis='x')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

#### Delivery Performance Analysis

In [ ]:
# Calculate delivery metrics
avg_delivery_time = bm.calculate_average_delivery_time(sales_current)
review_by_delivery = bm.calculate_review_by_delivery_speed(sales_current)

print(f"\nDelivery Performance - {CURRENT_START_YEAR}")
print("=" * 50)
print(f"Average Delivery Time: {avg_delivery_time:.1f} days")
print("\nAverage Review Score by Delivery Speed:")
print(review_by_delivery.to_string(index=False))
print("=" * 50)

In [ ]:
# Visualize delivery speed impact on reviews
plt.figure(figsize=(10, 6))
plt.bar(review_by_delivery['delivery_category'], 
        review_by_delivery['avg_review_score'],
        color=CHART_COLOR_PRIMARY)
plt.title(f'Average Review Score by Delivery Speed - {CURRENT_START_YEAR}', 
          fontsize=14, fontweight='bold', pad=20)
plt.xlabel('Delivery Speed', fontsize=12)
plt.ylabel('Average Review Score', fontsize=12)
plt.ylim(3.5, 5.0)
plt.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for i, v in enumerate(review_by_delivery['avg_review_score']):
    plt.text(i, v + 0.05, f'{v:.2f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

<a id='summary'></a>
## 6. Summary and Key Observations

This section summarizes the key findings from the analysis.

In [ ]:
# Generate comprehensive summary
summary_current = bm.generate_summary_statistics(sales_current, orders_current)

print("="*70)
print("BUSINESS PERFORMANCE SUMMARY")
print("="*70)
print(f"\nAnalysis Period: {CURRENT_START_YEAR}-{CURRENT_START_MONTH:02d} to {CURRENT_END_YEAR}-{CURRENT_END_MONTH:02d}")
print(f"Comparison Period: {COMPARISON_START_YEAR}-{COMPARISON_START_MONTH:02d} to {COMPARISON_END_YEAR}-{COMPARISON_END_MONTH:02d}")
print("\n" + "-"*70)
print("KEY METRICS")
print("-"*70)
print(f"Total Revenue: ${summary_current['total_revenue']:,.2f}")
print(f"Total Orders: {summary_current['total_orders']:,}")
print(f"Average Order Value: ${summary_current['average_order_value']:,.2f}")
print(f"Average Items per Order: {summary_current['average_items_per_order']:.2f}")
print(f"Average Review Score: {summary_current['average_review_score']:.2f}/5.00")
print(f"Average Delivery Time: {summary_current['average_delivery_days']:.1f} days")

print("\n" + "-"*70)
print("PERIOD-OVER-PERIOD COMPARISON")
print("-"*70)
print(f"Revenue Change: {comparison_metrics['revenue_growth_rate']*100:+.2f}%")
print(f"Orders Change: {comparison_metrics['orders_growth_rate']*100:+.2f}%")
print(f"AOV Change: {comparison_metrics['aov_growth_rate']*100:+.2f}%")

print("\n" + "-"*70)
print("TOP PERFORMERS")
print("-"*70)
print(f"Top Product Category: {category_revenue.iloc[0]['category']} (${category_revenue.iloc[0]['revenue']:,.2f})")
print(f"Top State by Revenue: {state_revenue.iloc[0]['state']} (${state_revenue.iloc[0]['revenue']:,.2f})")

print("\n" + "="*70)
print("\nKEY OBSERVATIONS:")
print("\n1. Revenue Performance:")
if comparison_metrics['revenue_growth_rate'] > 0:
    print(f"   - Revenue increased by {comparison_metrics['revenue_growth_rate']*100:.2f}% compared to the previous period")
else:
    print(f"   - Revenue declined by {abs(comparison_metrics['revenue_growth_rate'])*100:.2f}% compared to the previous period")

print(f"   - Average month-over-month growth: {avg_mom_growth*100:+.2f}%")

print("\n2. Customer Experience:")
print(f"   - Average customer satisfaction: {summary_current['average_review_score']:.2f}/5.00")
print(f"   - Delivery performance: {summary_current['average_delivery_days']:.1f} days average")
fastest_delivery = review_by_delivery.iloc[0]
print(f"   - {fastest_delivery['delivery_category']} deliveries achieve highest satisfaction ({fastest_delivery['avg_review_score']:.2f}/5.00)")

print("\n3. Product Performance:")
top_3_categories = category_revenue.head(3)
top_3_pct = (top_3_categories['revenue'].sum() / category_revenue['revenue'].sum()) * 100
print(f"   - Top 3 categories account for {top_3_pct:.1f}% of total revenue")
for idx, row in top_3_categories.iterrows():
    pct = (row['revenue'] / category_revenue['revenue'].sum()) * 100
    print(f"   - {row['category']}: {pct:.1f}% of revenue")

print("\n" + "="*70)

---

## Analysis Complete

This notebook can be re-run with different date ranges by modifying the configuration parameters in Section 3.